In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from scipy import stats

In [ ]:
coordinates_data_old = pd.read_feather("/mnt/upramdya_data/MD/F1_Tracks/Datasets/251104_09_F1_coordinates_F1_TNT_MB247_Data/summary/pooled_summary.feather")

In [4]:
coordinates_data_new = pd.read_feather("/mnt/upramdya_data/MD/F1_Tracks/F1_TNT/Datasets/260102_14_F1_coordinates_F1_TNT_Full_Data/summary/pooled_summary.feather")

In [5]:
coordinates_data_old.head()

,binned_slope_9,interaction_proportion,nb_significant_events,pulling_ratio,raw_pauses,interaction_rate_bin_7,final_event,has_major,binned_slope_8,interaction_rate_bin_1,...,Split,Date,Genotype,Period,FeedingState,Orientation,Light,Crossing,Pretraining,Unlocked
0,-0.024917,0.046835,2.0,0.0,[],0.000000,7.0,1.0,0.042533,0.010,...,n,251020,TNTxMB247,PM,starved_noWater,std,on,1,y,nn
1,0.029374,NaN,1.0,0.0,[],0.003374,NaN,1.0,0.000294,0.000,...,n,251020,TNTxMB247,PM,starved_noWater,std,on,1,y,nn
2,0.010802,0.103541,11.0,0.0,[],0.001667,17.0,1.0,0.045835,0.005,...,n,251020,TNTxMB247,PM,starved_noWater,std,on,1,y,nn
3,-0.000117,0.004763,1.0,0.0,[],0.000000,1.0,0.0,0.000197,0.000,...,n,251020,TNTxMB247,PM,starved_noWater,std,on,1,y,nn
4,0.009865,0.002223,0.0,NaN,[],0.000000,1.0,0.0,-0.013310,0.000,...,n,251020,TNTxMB247,PM,starved_noWater,std,on,1,y,yy


In [6]:
coordinates_data_new.head()

,major_event_first,interaction_rate_bin_9,binned_slope_11,interaction_rate_bin_10,final_event_time,overall_interaction_rate,binned_slope_2,interaction_rate_bin_5,binned_slope_5,auc,...,Period,FeedingState,Orientation,Light,Crossing,Pretraining,Unlocked,Balltype,Magnet,Exclude
0,False,0.033616,0.000026,0.040340,NaN,0.014028,-0.000008,0.016808,-0.000007,7386.952084,...,PM,starved_noWater,std,on,1,n,nn,NaN,NaN,None
1,False,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,...,PM,starved_noWater,std,on,1,n,nn,NaN,NaN,None
2,False,0.000000,NaN,0.025447,NaN,0.011250,NaN,0.047258,NaN,NaN,...,PM,starved_noWater,std,on,1,n,nn,NaN,NaN,None
3,False,0.000000,NaN,0.000000,98.896552,0.006805,NaN,0.000000,NaN,NaN,...,PM,starved_noWater,std,on,1,n,nn,NaN,NaN,None
4,False,0.023794,NaN,0.000000,NaN,0.010000,NaN,0.020395,NaN,NaN,...,PM,starved_noWater,std,on,1,n,nn,NaN,NaN,None


In [8]:
# check for ball_condition

coordinates_data_new['ball_identity'].value_counts()

ball_identity
test        237
training     97
Name: count, dtype: int64

In [9]:
# subset both datasets to include test ball only

old_test_data = coordinates_data_old[coordinates_data_old['ball_identity'] == 'test']
new_test_data = coordinates_data_new[coordinates_data_new['ball_identity'] == 'test']

In [10]:
# Select 1 random fly using "fly" column
flies = old_test_data['fly'].unique()

In [11]:
print(flies)

['251020_F1_TNT_Videos_Checked_arena2_Right'
 '251020_F1_TNT_Videos_Checked_arena2_Left'
 '251020_F1_TNT_Videos_Checked_arena7_Right'
 '251020_F1_TNT_Videos_Checked_arena7_Left'
 '251020_F1_TNT_Videos_Checked_arena6_Right'
 '251020_F1_TNT_Videos_Checked_arena6_Left'
 '251020_F1_TNT_Videos_Checked_arena5_Right'
 '251020_F1_TNT_Videos_Checked_arena5_Left'
 '251020_F1_TNT_Videos_Checked_arena8_Right'
 '251020_F1_TNT_Videos_Checked_arena8_Left'
 '251020_F1_TNT_Videos_Checked_arena3_Right'
 '251020_F1_TNT_Videos_Checked_arena3_Left'
 '251020_F1_TNT_Videos_Checked_arena4_Right'
 '251020_F1_TNT_Videos_Checked_arena4_Left'
 '251020_F1_TNT_Videos_Checked_arena1_Right'
 '251020_F1_TNT_Videos_Checked_arena1_Left'
 '251020_F1_TNT_Videos_Checked_arena9_Right'
 '251020_F1_TNT_Videos_Checked_arena9_Left'
 '251029_F1_TNT_1_Videos_Checked_arena2_Right'
 '251029_F1_TNT_1_Videos_Checked_arena2_Left'
 '251029_F1_TNT_1_Videos_Checked_arena7_Right'
 '251029_F1_TNT_1_Videos_Checked_arena7_Left'
 '251029_F1_T

In [16]:
# Get data for that fly from both datasets
fly_name = flies[28]
old_fly_data = old_test_data[old_test_data['fly'] == fly_name]
new_fly_data = new_test_data[new_test_data['fly'] == fly_name]

In [18]:
fly_name

'251029_F1_TNT_1_Videos_Checked_arena3_Left'

In [17]:
# Get distance_moved statistics for both datasets
old_distance_moved = old_fly_data['distance_moved']
new_distance_moved = new_fly_data['distance_moved']

print(f"Old Data - Mean: {old_distance_moved.mean()}, Std: {old_distance_moved.std()}")
print(f"New Data - Mean: {new_distance_moved.mean()}, Std: {new_distance_moved.std()}")

Old Data - Mean: 13.04608168468359, Std: nan
New Data - Mean: 65.78682616432754, Std: nan


In [20]:
# Load this fly using the ballpushing utils
# Add src to path
import sys
sys.path.append("/home/matthias/ballpushing_utils/src")
from Ballpushing_utils import Fly, Experiment, BallPushingMetrics

Fly_load = Fly("/mnt/upramdya_data/MD/F1_Tracks/Videos/251029_F1_TNT_1_Videos_Checked/arena3/Left", as_individual=True)

pygame 2.6.1 (SDL 2.28.4, Python 3.13.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [26]:
Fly_load.tracking_data.get_interactions_with_test_ball()

[[np.int64(108147), np.int64(108311), np.int64(165)],
 [np.int64(109935), np.int64(109999), np.int64(65)],
 [np.int64(110092), np.int64(110278), np.int64(187)],
 [np.int64(110310), np.int64(110340), np.int64(31)],
 [np.int64(110420), np.int64(110590), np.int64(171)],
 [np.int64(110627), np.int64(110824), np.int64(198)],
 [np.int64(111850), np.int64(112655), np.int64(806)],
 [np.int64(113174), np.int64(113449), np.int64(276)],
 [np.int64(113978), np.int64(114027), np.int64(50)],
 [np.int64(118607), np.int64(118637), np.int64(31)],
 [np.int64(119035), np.int64(119133), np.int64(99)],
 [np.int64(119490), np.int64(119528), np.int64(39)],
 [np.int64(120004), np.int64(120094), np.int64(91)],
 [np.int64(120136), np.int64(120201), np.int64(66)],
 [np.int64(120288), np.int64(120397), np.int64(110)],
 [np.int64(120791), np.int64(120846), np.int64(56)],
 [np.int64(121015), np.int64(121066), np.int64(52)],
 [np.int64(121148), np.int64(121182), np.int64(35)],
 [np.int64(121475), np.int64(121510), n